In [3]:
delete_substr = ['#', '<user>', '<url>']
dataset.create_count('../data/train_pos_full.txt', delete_substr, True,0)
dataset.create_count('../data/train_neg_full.txt', delete_substr, False,0)

Total lines (label = True): 1250000
10000 lines processed
20000 lines processed
30000 lines processed
40000 lines processed
50000 lines processed
60000 lines processed
70000 lines processed
80000 lines processed
90000 lines processed
100000 lines processed
110000 lines processed
120000 lines processed
130000 lines processed
140000 lines processed
150000 lines processed
160000 lines processed
170000 lines processed
180000 lines processed
190000 lines processed
200000 lines processed
210000 lines processed
220000 lines processed
230000 lines processed
240000 lines processed
250000 lines processed
260000 lines processed
270000 lines processed
280000 lines processed
290000 lines processed
300000 lines processed
310000 lines processed
320000 lines processed
330000 lines processed
340000 lines processed
350000 lines processed
360000 lines processed
370000 lines processed
380000 lines processed
390000 lines processed
400000 lines processed
410000 lines processed
420000 lines processed
430000 

In [6]:
dataset.pos_count=np.load('pos_count.npy')
dataset.neg_count=np.load('neg_count.npy')

In [5]:
np.save('pos_count', dataset.pos_count)
np.save('neg_count', dataset.neg_count)

In [8]:
dataset.neg_count['believe']/dataset.pos_count['believe']

1.2881218274111674

In [4]:
pos_count=np.load('pos_count.npy')
neg_count=np.load('neg_count.npy')

# CNN & LSTM

In [37]:
%matplotlib inline
import numpy as np
from sklearn.model_selection import train_test_split
from create_submission import create_csv_submission
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import keras
import matplotlib.pyplot as plt
import Dataset as data
from gensim.models import Word2Vec
from Word2VecModel import Word2VecModel
from nltk.corpus import stopwords
from Dataset import DataSet

from keras.layers.merge import concatenate
from keras.models import Model
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D,Flatten
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import BatchNormalization


In [4]:
word2vec = Word2VecModel(vector_size=128, word_min_count=5)
word2vec.load_model('model_word2vec_128.bin')
dataset = DataSet(word2vec)

In [5]:
deleted_substr = ['#', '<user>', '<url>']
dataset.create_train_test('../data/train_pos_full.txt', '../data/train_neg_full.txt', delete_substr=deleted_substr,\
                         training=True, cnn=True,weight=0, stop_words=True)

Total lines (): 1250000
10000 lines processed  len words 4
20000 lines processed  len words 3
30000 lines processed  len words 4
40000 lines processed  len words 2
50000 lines processed  len words 11
60000 lines processed  len words 6
70000 lines processed  len words 10
80000 lines processed  len words 1
90000 lines processed  len words 9
100000 lines processed  len words 10
110000 lines processed  len words 5
120000 lines processed  len words 3
130000 lines processed  len words 18
140000 lines processed  len words 8
150000 lines processed  len words 10
160000 lines processed  len words 11
170000 lines processed  len words 5
180000 lines processed  len words 2
190000 lines processed  len words 3
200000 lines processed  len words 1
210000 lines processed  len words 6
220000 lines processed  len words 6
230000 lines processed  len words 4
240000 lines processed  len words 10
250000 lines processed  len words 2
260000 lines processed  len words 8
270000 lines processed  len words 4
280000

1010000 lines processed  len words 11
1020000 lines processed  len words 7
1030000 lines processed  len words 14
1040000 lines processed  len words 4
1050000 lines processed  len words 5
1060000 lines processed  len words 5
1070000 lines processed  len words 10
1080000 lines processed  len words 5
1090000 lines processed  len words 3
1100000 lines processed  len words 10
1110000 lines processed  len words 6
1120000 lines processed  len words 11
1130000 lines processed  len words 3
1140000 lines processed  len words 6
1150000 lines processed  len words 14
1160000 lines processed  len words 10
1170000 lines processed  len words 8
1180000 lines processed  len words 11
1190000 lines processed  len words 15
1200000 lines processed  len words 13
1210000 lines processed  len words 5
1220000 lines processed  len words 5
1230000 lines processed  len words 7
1240000 lines processed  len words 9
1250000 lines processed  len words 13
X_train shape: (2490064,)
y_train shape: (2490064,)


In [6]:
dataset.X_train_cnn = np.load('X_train_cnn.npy')

In [7]:
dataset.y_train_cnn = np.load('y_train_cnn.npy')

In [14]:
dataset.X_train_cnn[6]

['feel',
 'rollercoaster',
 'life',
 'song',
 'life',
 'yolo',
 'becoming',
 'famous',
 'x']

In [8]:
dataset.X_test_cnn=[]
deleted_substr = ['#', '<user>', '<url>']
dataset.create_train_test('../data/test_data.txt', '', delete_substr=deleted_substr, training=False, \
                           cnn=True,weight=0, stop_words=True)

Total lines (): 10000
10000 lines processed  len words 16
X_test shape: (10000,)


In [9]:
dataset.X_test_cnn = np.load('X_test_cnn.npy')

In [10]:
a=[len(line) for line in dataset.X_train_cnn]

In [13]:
np.median(a)

7.0

In [30]:
train_data, test_data, embedding_layer = dataset.create_embedding(embedding_dim=128,seq_length=40)

Null word embeddings: 16672


In [31]:
X_train, X_valid, y_train, y_valid = train_test_split(train_data, dataset.y_train_cnn, test_size=0.2, random_state=23)

In [32]:
conv_filters = 128

inp = Input(shape=(X_train.shape[1],))
emb = embedding_layer(inp)

conv1_1 = Conv1D(filters=conv_filters, kernel_size=3)(emb)
btch1_1 = BatchNormalization()(conv1_1)
drp1_1  = Dropout(0.2)(btch1_1)
actv1_1 = Activation('relu')(drp1_1)
glmp1_1 = GlobalMaxPooling1D()(actv1_1)

conv1_2 = Conv1D(filters=conv_filters, kernel_size=4)(emb)
btch1_2 = BatchNormalization()(conv1_2)
drp1_2  = Dropout(0.2)(btch1_2)
actv1_2 = Activation('relu')(drp1_2)
glmp1_2 = GlobalMaxPooling1D()(actv1_2)

# Gather all convolution layers
cnct = concatenate([glmp1_1, glmp1_2], axis=1)
drp1 = Dropout(0.2)(cnct)

dns1  = Dense(32, activation='relu')(drp1)
btch1 = BatchNormalization()(dns1)
drp2  = Dropout(0.2)(btch1)

out = Dense(1, activation='sigmoid')(drp2)
model2 = Model(inputs=inp, outputs=out)
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 15, 128)      12622080    input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 13, 128)      49280       embedding_2[1][0]                
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 12, 128)      65664       embedding_2[1][0]                
__________________________________________________________________________________________________
batch_norm

In [33]:
model2.fit(X_train, y_train,  validation_split=0.1, epochs=4, shuffle=True, verbose=1,batch_size=128)

Train on 1793242 samples, validate on 199250 samples
Epoch 1/4
1793242/1793242 [==============================] - 306s 170us/step - loss: 0.4416 - acc: 0.7848 - val_loss: 0.4128 - val_acc: 0.8047
Epoch 2/4
 384640/1793242 [=====>........................] - ETA: 4:03 - loss: 0.4215 - acc: 0.7985

KeyboardInterrupt: 

In [39]:
model_conv = Sequential()
model_conv.add(embedding_layer)
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(100))
model_conv.add(Dense(1, activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])

In [40]:
model_conv.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 128)           12622080  
_________________________________________________________________
dropout_8 (Dropout)          (None, 40, 128)           0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 36, 64)            41024     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 9, 64)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 101       
Total params: 12,729,205
Trainable params: 107,125
Non-trainable params: 12,622,080
__________________________________________________________

In [41]:
model_conv.fit(X_train, y_train,  validation_split=0.1, epochs=4, shuffle=True, verbose=1,batch_size=128)

Train on 1792845 samples, validate on 199206 samples
Epoch 1/4
1792845/1792845 [==============================] - 383s 214us/step - loss: 0.4363 - acc: 0.7863 - val_loss: 0.4127 - val_acc: 0.8015
Epoch 2/4
  44160/1792845 [..............................] - ETA: 6:30 - loss: 0.4197 - acc: 0.7956

KeyboardInterrupt: 

In [46]:
inputs = Input(shape=(X_train.shape[1],))
layer = embedding_layer(inputs)
layer = LSTM(64)(layer)
layer = Dense(1,name='out_layer')(layer)
layer = Activation('sigmoid')(layer)
model = Model(inputs=inputs,outputs=layer)
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [47]:
model.fit(X_train, y_train,  validation_split=0.1, epochs=4, shuffle=True, verbose=1,batch_size=128)

Train on 1792845 samples, validate on 199206 samples
Epoch 1/4
1792845/1792845 [==============================] - 530s 295us/step - loss: 0.4037 - acc: 0.8079 - val_loss: 0.3914 - val_acc: 0.8154
Epoch 2/4
1792845/1792845 [==============================] - 594s 331us/step - loss: 0.3826 - acc: 0.8209 - val_loss: 0.3858 - val_acc: 0.8192
Epoch 3/4
1792845/1792845 [==============================] - 527s 294us/step - loss: 0.3753 - acc: 0.8252 - val_loss: 0.3828 - val_acc: 0.8213
Epoch 4/4
1792845/1792845 [==============================] - 518s 289us/step - loss: 0.3705 - acc: 0.8277 - val_loss: 0.3811 - val_acc: 0.8216


In [45]:
model.fit(X_train, y_train,  validation_split=0.1, epochs=4, shuffle=True, verbose=1,batch_size=128)

Train on 1792845 samples, validate on 199206 samples
Epoch 1/4
1792845/1792845 [==============================] - 507s 283us/step - loss: 0.3684 - acc: 0.8291 - val_loss: 0.3813 - val_acc: 0.8225
Epoch 2/4
1792845/1792845 [==============================] - 508s 283us/step - loss: 0.3659 - acc: 0.8303 - val_loss: 0.3798 - val_acc: 0.8233
Epoch 3/4
1792845/1792845 [==============================] - 509s 284us/step - loss: 0.3638 - acc: 0.8316 - val_loss: 0.3797 - val_acc: 0.8231
Epoch 4/4
1792845/1792845 [==============================] - 510s 285us/step - loss: 0.3617 - acc: 0.8326 - val_loss: 0.3793 - val_acc: 0.8231


In [32]:
model_1 = Sequential()
model_1.add(embedding_layer)
model_1.add(Conv1D(250,5,padding='valid',activation='relu',strides=1))
model_1.add(GlobalMaxPooling1D())
model_1.add(Dense(250))
#model_1.add(Dropout(0.4))
model_1.add(Activation('relu'))
model_1.add(Dense(1))
model_1.add(Activation('sigmoid'))
model_1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 40, 128)           12622080  
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 36, 250)           160250    
_________________________________________________________________
global_max_pooling1d_5 (Glob (None, 250)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 250)               62750     
_________________________________________________________________
activation_9 (Activation)    (None, 250)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 251       
_________________________________________________________________
activation_10 (Activation)   (None, 1)                 0         
Total para

In [33]:
model_1.fit(X_train, y_train,  validation_split=0.1, epochs=4, shuffle=True, verbose=1,batch_size=128)

Train on 1792845 samples, validate on 199206 samples
Epoch 1/4
1638784/1792845 [==========================>...] - ETA: 58s - loss: 0.4178 - acc: 0.7989

KeyboardInterrupt: 

In [35]:
model_1.fit(X_train, y_train,  validation_split=0.1, epochs=4, shuffle=True, verbose=1,batch_size=128)

Train on 1793242 samples, validate on 199250 samples
Epoch 1/4
1793242/1793242 [==============================] - 225s 125us/step - loss: 0.4219 - acc: 0.7965 - val_loss: 0.4032 - val_acc: 0.8076
Epoch 2/4
 989056/1793242 [===============>..............] - ETA: 1:35 - loss: 0.3960 - acc: 0.8124

KeyboardInterrupt: 